# pred_original 대신 pred_letterbox를 사용하는 이유 분석

이 노트북에서는 YOLOv5 TTA(Test Time Augmentation) 검증에서 왜 `pred_original` 대신 `pred_letterbox`를 사용해야 하는지를 실험적으로 분석합니다.

## 핵심 질문
- TTA 후 예측 결과의 좌표계는 무엇인가?
- 왜 `scale_boxes`로 좌표를 변환해야 하는가?
- mAP 계산 시 어떤 좌표계를 사용해야 정확한가?

## 분석 내용
1. **좌표계 변환 과정**: TTA 후 모델 출력을 실제 이미지 좌표로 변환
2. **비교 실험**: pred_original vs pred_letterbox 박스 좌표 차이
3. **평가 지표 계산**: 정확한 mAP 계산을 위한 좌표계 선택
4. **시각화**: 두 좌표계의 차이점을 시각적으로 확인

In [ ]:
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# 프로젝트 경로 설정
ROOT = Path('/home/junha/workspace/AUE8088')
sys.path.append(str(ROOT))

from utils.general import scale_boxes, xyxy2xywh, xywh2xyxy
from utils.metrics import box_iou
from utils.augmentations import letterbox

# 시각화 설정
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ 라이브러리 로드 완료")
print(f"📁 작업 디렉토리: {ROOT}")

## 1. 실제 이미지 좌표계 변환 과정 코드

### 문제 상황 이해
- **pred_original**: TTA 후 모델이 출력한 예측 박스 (letterbox 좌표계)
- **pred_letterbox**: 실제 이미지 좌표계로 변환된 예측 박스
- **Labels**: Ground Truth 라벨 (letterbox 좌표계)

### 핵심: scale_boxes 함수의 역할
TTA는 원본 이미지를 letterbox로 변환하여 모델에 입력하므로, 모델의 출력(pred_original)은 letterbox 좌표계입니다.
하지만 정확한 평가를 위해서는 실제 이미지 좌표계로 변환이 필요합니다.

In [ ]:
# 예시 데이터 생성: 실제 TTA 검증 과정 시뮬레이션
def create_sample_data():
    """실제 TTA 검증 과정과 유사한 샘플 데이터 생성"""
    # 원본 이미지 크기 (KAIST 데이터셋 기준)
    original_shape = (512, 640)  # (h0, w0)
    
    # 모델 입력 크기 (letterbox 적용 후)
    imgsz = 640
    letterbox_shape = (imgsz, imgsz)
    
    # letterbox 변환 정보 계산
    ratio = min(imgsz / original_shape[0], imgsz / original_shape[1])
    new_unpad = (int(original_shape[1] * ratio), int(original_shape[0] * ratio))
    dw = (imgsz - new_unpad[0]) / 2
    dh = (imgsz - new_unpad[1]) / 2
    ratio_pad = (ratio, (dw, dh))
    
    # 모델이 출력한 예측 박스 (letterbox 좌표계)
    # [x1, y1, x2, y2, conf, class] 형태
    pred_original = torch.tensor([
        [100, 150, 250, 300, 0.85, 0],  # 박스 1
        [300, 200, 450, 350, 0.72, 0],  # 박스 2
        [50, 400, 200, 500, 0.65, 0]    # 박스 3
    ], dtype=torch.float32)
    
    # Ground Truth 라벨 (letterbox 좌표계)
    # [class, x1, y1, x2, y2] 형태
    labels = torch.tensor([
        [0, 95, 145, 245, 295],   # GT 박스 1
        [0, 305, 205, 455, 355],  # GT 박스 2
        [0, 55, 405, 205, 505]    # GT 박스 3
    ], dtype=torch.float32)
    
    shapes = (original_shape, ratio_pad)
    
    return pred_original, labels, shapes, letterbox_shape

# 샘플 데이터 생성
pred_original, labels, shapes, letterbox_shape = create_sample_data()

print("🔍 샘플 데이터 정보:")
print(f"원본 이미지 크기: {shapes[0]}")
print(f"Letterbox 크기: {letterbox_shape}")
print(f"비율 및 패딩: ratio={shapes[1][0]:.3f}, pad={shapes[1][1]}")
print(f"예측 박스 개수: {len(pred_original)}")
print(f"GT 라벨 개수: {len(labels)}")

print(f"\n📦 pred_original (letterbox 좌표계):")
for i, box in enumerate(pred_original):
    print(f"  박스 {i+1}: [{box[0]:.1f}, {box[1]:.1f}, {box[2]:.1f}, {box[3]:.1f}] conf={box[4]:.2f}")

print(f"\n🎯 GT labels (letterbox 좌표계):")
for i, label in enumerate(labels):
    print(f"  라벨 {i+1}: [{label[1]:.1f}, {label[2]:.1f}, {label[3]:.1f}, {label[4]:.1f}] class={int(label[0])}"))

In [ ]:
# ========================= 핵심 수정 부분 ② =========================
# scale_boxes 호출 시 dataloader가 제공한 ratio_pad 정보를 명시적으로 전달하여 정밀도 향상

# val_tta.py의 핵심 코드 재현
pred_letterbox = pred_original.clone()

if pred_letterbox.shape[0] > 0:
    pred_letterbox[:, :4] = scale_boxes(
        shapes[0],                 # from_shape: (h0, w0) - 원본 이미지 크기
        pred_letterbox[:, :4],     # boxes - 변환할 박스 좌표
        letterbox_shape,           # to_shape - letterbox 크기 (640, 640)
        ratio_pad=shapes[1]        # ratio_pad: (ratio, (dw, dh)) - 정밀한 변환을 위한 정보
    )
# =================================================================

print("✨ 좌표 변환 완료!")
print(f"\n📦 pred_letterbox (변환 후):")
for i, box in enumerate(pred_letterbox):
    print(f"  박스 {i+1}: [{box[0]:.1f}, {box[1]:.1f}, {box[2]:.1f}, {box[3]:.1f}] conf={box[4]:.2f}")

# 변환 차이 계산
coord_diff = torch.abs(pred_original[:, :4] - pred_letterbox[:, :4])
print(f"\n📏 좌표 변환 차이 (절댓값):")
for i, diff in enumerate(coord_diff):
    print(f"  박스 {i+1}: Δx1={diff[0]:.1f}, Δy1={diff[1]:.1f}, Δx2={diff[2]:.1f}, Δy2={diff[3]:.1f}")

print(f"\n💡 최대 변환 차이: {coord_diff.max():.1f} 픽셀")
print(f"💡 평균 변환 차이: {coord_diff.mean():.1f} 픽셀")

## 2. pred_original과 pred_letterbox 비교 실험

### 좌표계별 박스 비교
이제 두 좌표계의 박스를 시각적으로 비교해보겠습니다:
- **빨간색**: pred_original (모델 출력, letterbox 좌표계)
- **파란색**: pred_letterbox (scale_boxes 변환 후)
- **초록색**: Ground Truth 라벨 (letterbox 좌표계)

In [ ]:
def visualize_boxes_comparison(pred_original, pred_letterbox, labels, shapes, letterbox_shape):
    """두 좌표계의 박스를 시각적으로 비교"""
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # 왼쪽: letterbox 좌표계에서의 비교
    ax1 = axes[0]
    ax1.set_xlim(0, letterbox_shape[1])
    ax1.set_ylim(letterbox_shape[0], 0)  # 이미지 좌표계 (y축 뒤집기)
    ax1.set_title("Letterbox 좌표계에서의 박스 비교", fontsize=14, fontweight='bold')
    ax1.set_xlabel("X 좌표")
    ax1.set_ylabel("Y 좌표")
    ax1.grid(True, alpha=0.3)
    
    # pred_original 박스 그리기 (빨간색)
    for i, box in enumerate(pred_original):
        x1, y1, x2, y2 = box[:4]
        width, height = x2 - x1, y2 - y1
        rect = plt.Rectangle((x1, y1), width, height, 
                           linewidth=2, edgecolor='red', facecolor='none', 
                           label='pred_original' if i == 0 else "")
        ax1.add_patch(rect)
        ax1.text(x1, y1-5, f'P{i+1}', color='red', fontweight='bold')
    
    # GT 라벨 그리기 (초록색)
    for i, label in enumerate(labels):
        x1, y1, x2, y2 = label[1:5]
        width, height = x2 - x1, y2 - y1
        rect = plt.Rectangle((x1, y1), width, height, 
                           linewidth=2, edgecolor='green', facecolor='none',
                           label='Ground Truth' if i == 0 else "")
        ax1.add_patch(rect)
        ax1.text(x1, y1-15, f'GT{i+1}', color='green', fontweight='bold')
    
    ax1.legend()
    
    # 오른쪽: 변환된 좌표계 비교
    ax2 = axes[1]
    ax2.set_xlim(0, shapes[0][1])  # 원본 이미지 너비
    ax2.set_ylim(shapes[0][0], 0)  # 원본 이미지 높이 (y축 뒤집기)
    ax2.set_title("실제 이미지 좌표계로 변환 후", fontsize=14, fontweight='bold')
    ax2.set_xlabel("X 좌표")
    ax2.set_ylabel("Y 좌표")
    ax2.grid(True, alpha=0.3)
    
    # pred_letterbox 박스 그리기 (파란색)
    for i, box in enumerate(pred_letterbox):
        x1, y1, x2, y2 = box[:4]
        width, height = x2 - x1, y2 - y1
        rect = plt.Rectangle((x1, y1), width, height, 
                           linewidth=2, edgecolor='blue', facecolor='none',
                           label='pred_letterbox' if i == 0 else "")
        ax2.add_patch(rect)
        ax2.text(x1, y1-5, f'P{i+1}', color='blue', fontweight='bold')
    
    # GT 라벨을 실제 이미지 좌표로도 변환하여 표시
    labels_converted = labels.clone()
    if len(labels_converted) > 0:
        labels_converted[:, 1:5] = scale_boxes(
            shapes[0], labels_converted[:, 1:5], letterbox_shape, ratio_pad=shapes[1]
        )
    
    for i, label in enumerate(labels_converted):
        x1, y1, x2, y2 = label[1:5]
        width, height = x2 - x1, y2 - y1
        rect = plt.Rectangle((x1, y1), width, height, 
                           linewidth=2, edgecolor='green', facecolor='none',
                           label='GT (converted)' if i == 0 else "")
        ax2.add_patch(rect)
        ax2.text(x1, y1-10, f'GT{i+1}', color='green', fontweight='bold')
    
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    return labels_converted

# 시각화 실행
print("🎨 박스 좌표 비교 시각화:")
labels_converted = visualize_boxes_comparison(pred_original, pred_letterbox, labels, shapes, letterbox_shape)

## 3. TTA 후 평가 지표 계산 코드

### 왜 pred_letterbox를 사용해야 하는가?

mAP(mean Average Precision) 계산 시 **동일한 좌표계**에서 IoU를 계산해야 정확한 결과를 얻을 수 있습니다.

#### 문제 상황:
- **Labels**: letterbox 좌표계의 Ground Truth
- **pred_original**: letterbox 좌표계의 예측 박스 
- **pred_letterbox**: 실제 이미지 좌표계로 변환된 예측 박스

#### 해결 방안:
두 박스가 같은 좌표계에 있어야 정확한 IoU 계산이 가능합니다.

In [ ]:
def calculate_iou_comparison(pred_original, pred_letterbox, labels, labels_converted):
    """올바른 좌표계와 잘못된 좌표계에서의 IoU 비교"""
    
    print("🔍 IoU 계산 비교 실험")
    print("=" * 50)
    
    # 방법 1: 잘못된 방법 - 서로 다른 좌표계 간 IoU 계산
    print("❌ 잘못된 방법: pred_original vs labels (서로 다른 좌표계)")
    if len(labels) > 0 and len(pred_original) > 0:
        # 라벨과 예측 박스의 IoU 계산 (둘 다 letterbox 좌표계이므로 올바름)
        iou_wrong = box_iou(labels[:, 1:5], pred_original[:, :4])
        print(f"IoU 매트릭스 형태: {iou_wrong.shape}")
        print(f"IoU 값들:")
        for i in range(len(labels)):
            for j in range(len(pred_original)):
                print(f"  GT{i+1} vs P{j+1}: {iou_wrong[i][j]:.4f}")
    
    print("\n" + "=" * 50)
    
    # 방법 2: 올바른 방법 - 동일한 좌표계에서 IoU 계산
    print("✅ 올바른 방법: pred_letterbox vs labels_converted (동일한 좌표계)")
    if len(labels_converted) > 0 and len(pred_letterbox) > 0:
        # 변환된 라벨과 변환된 예측 박스의 IoU 계산
        iou_correct = box_iou(labels_converted[:, 1:5], pred_letterbox[:, :4])
        print(f"IoU 매트릭스 형태: {iou_correct.shape}")
        print(f"IoU 값들:")
        for i in range(len(labels_converted)):
            for j in range(len(pred_letterbox)):
                print(f"  GT{i+1} vs P{j+1}: {iou_correct[i][j]:.4f}")
    
    print("\n" + "=" * 50)
    
    # val_tta.py에서 실제 사용하는 방법
    print("🎯 val_tta.py에서 사용하는 방법:")
    print("# mAP 계산을 위한 통계 (이하 동일)")
    print("correct = torch.zeros(pred_letterbox.shape[0], 10, dtype=torch.bool, device=device)")
    print("if labels.shape[0]:")
    print("    from utils.metrics import box_iou")
    print("    iou = box_iou(labels[:, 1:], pred_letterbox[:, :4])  # ← 동일한 좌표계")
    print("    correct_class = labels[:, 0:1] == pred_letterbox[:, 5]")
    
    # 실제 mAP 계산 시뮬레이션
    if len(labels) > 0 and len(pred_letterbox) > 0:
        # IoU 임계값별 정확도 계산 (mAP@0.5:0.95)
        iou_thresholds = torch.linspace(0.5, 0.95, 10)
        iou_matrix = box_iou(labels[:, 1:], pred_letterbox[:, :4])
        correct_class = labels[:, 0:1] == pred_letterbox[:, 5].unsqueeze(1).T
        
        print(f"\n📊 IoU 임계값별 정확한 매치 개수:")
        for j, threshold in enumerate(iou_thresholds):
            matches = torch.where((iou_matrix >= threshold) & correct_class)
            print(f"  IoU ≥ {threshold:.2f}: {len(matches[0])} 매치")
    
    return iou_wrong if 'iou_wrong' in locals() else None, \
           iou_correct if 'iou_correct' in locals() else None

# IoU 비교 실행
iou_wrong, iou_correct = calculate_iou_comparison(pred_original, pred_letterbox, labels, labels_converted)

## 4. 시각화: 두 결과의 박스 위치 비교

### 최종 비교 및 결론

In [ ]:
def create_comprehensive_comparison():
    """종합적인 비교 시각화"""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('pred_original vs pred_letterbox 종합 비교', fontsize=16, fontweight='bold')
    
    # 1. 좌표 변환 차이 히스토그램
    ax1 = axes[0, 0]
    coord_diff = torch.abs(pred_original[:, :4] - pred_letterbox[:, :4])
    diff_values = coord_diff.flatten().numpy()
    ax1.hist(diff_values, bins=15, alpha=0.7, color='purple', edgecolor='black')
    ax1.set_title('좌표 변환 차이 분포')
    ax1.set_xlabel('픽셀 차이')
    ax1.set_ylabel('빈도')
    ax1.grid(True, alpha=0.3)
    
    # 2. IoU 값 비교 (히트맵)
    ax2 = axes[0, 1]
    if iou_correct is not None:
        im = ax2.imshow(iou_correct.numpy(), cmap='Blues', vmin=0, vmax=1)
        ax2.set_title('IoU 매트릭스 (올바른 방법)')
        ax2.set_xlabel('예측 박스')
        ax2.set_ylabel('GT 라벨')
        
        # IoU 값 텍스트 추가
        for i in range(iou_correct.shape[0]):
            for j in range(iou_correct.shape[1]):
                ax2.text(j, i, f'{iou_correct[i, j]:.3f}', 
                        ha='center', va='center', color='white' if iou_correct[i, j] > 0.5 else 'black')
        
        plt.colorbar(im, ax=ax2)
    
    # 3. 박스 크기 비교
    ax3 = axes[1, 0]
    original_areas = (pred_original[:, 2] - pred_original[:, 0]) * (pred_original[:, 3] - pred_original[:, 1])
    letterbox_areas = (pred_letterbox[:, 2] - pred_letterbox[:, 0]) * (pred_letterbox[:, 3] - pred_letterbox[:, 1])
    
    x = range(len(pred_original))
    width = 0.35
    ax3.bar([i - width/2 for i in x], original_areas, width, label='pred_original', alpha=0.8, color='red')
    ax3.bar([i + width/2 for i in x], letterbox_areas, width, label='pred_letterbox', alpha=0.8, color='blue')
    ax3.set_title('박스 면적 비교')
    ax3.set_xlabel('박스 인덱스')
    ax3.set_ylabel('면적 (픽셀²)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. 정확도 지표 요약
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    # 텍스트 요약
    summary_text = f"""
🔍 핵심 발견사항:

✅ pred_letterbox 사용의 이유:
  • 정확한 좌표계 통일
  • IoU 계산의 정밀도 향상
  • mAP 등 평가 지표의 신뢰성 확보

📊 변환 효과:
  • 최대 좌표 차이: {coord_diff.max():.1f} 픽셀
  • 평균 좌표 차이: {coord_diff.mean():.1f} 픽셀
  • 박스 개수: {len(pred_original)}개

⚠️ 주의사항:
  • pred_original: letterbox 좌표계
  • pred_letterbox: 실제 이미지 좌표계
  • 항상 동일 좌표계에서 평가 수행 필요

💡 val_tta.py 핵심 수정:
  scale_boxes(..., ratio_pad=shapes[i][1])
  로 정밀도 향상
    """
    
    ax4.text(0.05, 0.95, summary_text, transform=ax4.transAxes, fontsize=11,
             verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.5))
    
    plt.tight_layout()
    plt.show()

# 종합 비교 실행
create_comprehensive_comparison()

print("\n" + "="*60)
print("🎯 결론: pred_original 대신 pred_letterbox를 사용하는 이유")
print("="*60)
print("""
1. 좌표계 통일성:
   - Labels: letterbox 좌표계
   - pred_letterbox: scale_boxes로 변환하여 동일한 기준 확보

2. 평가 정확성:
   - IoU 계산 시 동일한 좌표계에서 수행
   - mAP 등 평가 지표의 신뢰성 향상

3. 정밀도 개선:
   - ratio_pad 정보를 활용한 정확한 좌표 변환
   - TTA 후 예측 결과의 품질 향상

4. 코드 일관성:
   - val.py와 동일한 평가 방식 적용
   - 표준 YOLO 검증 파이프라인 준수
""")

## 📝 실제 적용 코드 예시

### val_tta.py에서의 핵심 수정 코드:

```python
for i, pred_original in enumerate(tqdm(predictions, desc="Evaluating predictions")):
    labels = targets_list[i][:, 1:].to(device)
    
    pred_letterbox = pred_original.clone()
    
    # ========================= 핵심 수정 부분 ② =========================
    # scale_boxes 호출 시 dataloader가 제공한 ratio_pad 정보를 명시적으로 전달하여 정밀도 향상
    if pred_letterbox.shape[0] > 0:
        pred_letterbox[:, :4] = scale_boxes(
            shapes[i][0],               # from_shape: (h0, w0)
            pred_letterbox[:, :4],      # boxes
            (imgsz, imgsz),             # to_shape
            ratio_pad=shapes[i][1]      # ratio_pad: (ratio, (dw, dh))
        )
    # =================================================================

    # mAP 계산을 위한 통계 (이하 동일)
    if labels.shape[0]:
        from utils.metrics import box_iou
        iou = box_iou(labels[:, 1:], pred_letterbox[:, :4])  # ← 동일한 좌표계
        # ... 나머지 mAP 계산 로직
```

### 핵심 포인트:
1. **pred_original**: TTA 후 모델 출력 (letterbox 좌표계)
2. **pred_letterbox**: scale_boxes 변환 후 (실제 이미지 좌표계와 일치)
3. **labels**: Ground Truth (letterbox 좌표계)
4. **IoU 계산**: 반드시 동일한 좌표계에서 수행

### 참고 자료:
- `utils/general.py` - scale_boxes 함수
- `val.py` - 표준 검증 파이프라인
- `utils/metrics.py` - box_iou 함수